In [10]:
# Changing the working directory
import os
os.chdir('../')

import gspread
from utils import *
from skydb.sheets import updateSpreadsheet, createSpreadsheet

In [2]:
records = getOfficialRecords()
grades = records['grades']
gpas = records['gpas']
students = getStudents().drop(['birth_date'], axis=1)
psat_scores = getPsatScores()

In [3]:
""" PSAT Scores """
math_psat = psat_scores[['user_id', 'math_score']]
psat_students = (
    students
    .query("""grade_level in [9,10,11]""")
    .drop('counselor', axis=1)
    .rename(columns={'id':'user_id'})
    .astype({'user_id':'int64', 'student_id':'float', 'grade_level':'int64'})
    .merge(math_psat,
                'left',
                on='user_id'
    )
)

In [4]:
""" Algebra 1/2 Grades """
raw_math_grades = grades[grades.course_title.str.contains('Algebra \\d')].copy()
for index, row in raw_math_grades.iterrows():
    if row['school_year'] == '2021 - 2022' and row['term'] == 'Fall':
        raw_math_grades.loc[index, 'official_grade'] = row['grade']
    elif row['term'] == 'Spring':
        raw_math_grades.loc[index, 'official_grade'] = row['grade']
    else:
        raw_math_grades.loc[index, 'official_grade'] = None
        
math_grades = raw_math_grades[~raw_math_grades.official_grade.isna()].copy().drop('grade', axis=1)

raw_math_students = (
    psat_students
    .merge(math_grades,
           'inner',
           on='user_id'
    )
)

""" Label Alegebra Classes"""
for index, row in raw_math_students.iterrows():
    raw_math_students.loc[index, 'label'] = 'Algebra 1' if 'Algebra 1' in row['course_title'] else 'Algebra 2'
    
""" Spread Data """
clean_math_students = (
    raw_math_students
    .groupby(['user_id','label'])
    .official_grade
    .first()
    .unstack()
)

""" Joining with student Data """
math_students = (
    psat_students
    .merge(clean_math_students,
           'inner',
           on='user_id'
    )
)

In [5]:
math_students

,user_id,student_id,first_name,last_name,preferred_name,grade_level,math_score,Algebra 1,Algebra 2
0,4748327,6078.0,Emillie,Wise,Em,9,420.0,C-,NaN
1,4748339,6063.0,Patrick,Fox,Patrick,10,530.0,A-,A+
2,4748342,6075.0,Tyler,Sharon,Tyler,11,550.0,NaN,D+
3,4748354,6084.0,Cooper,Pickering,Cooper,10,590.0,NaN,A-
4,4748363,6085.0,Enzo,Bezerra,Enzo,10,NaN,C-,NaN
...,...,...,...,...,...,...,...,...,...
289,4905718,6203.0,Simon,Matthews,Simon,9,640.0,NaN,A
290,4923164,6159.0,Grace,Flury,Grace,10,390.0,NaN,B+
291,4931681,6284.0,Audrey,Kassewitz,Audrey,10,510.0,NaN,A-
292,4933183,6282.0,Johnathan,Lavender,Johnathan,9,520.0,B+,NaN


In [6]:
""" Cumulative GPA """
math_data = (
    math_students
    .merge(gpas,
           'inner',
           on='user_id'
    )
    .sort_values(['last_name', 'first_name'])
    .drop(['user_id', 'student_id', 'first_name'], axis=1)
)

In [7]:
# updateSpreadsheet(math_data.fillna(''), 
#                   sheet_id='1xgA6DrPoU3QHGBMZVichqyikiA3X82C8yiUHn1QnxFE',
#                   styleClass=HysonFireStyle
#                  )

In [ ]:
# Tableau Data 
updateSpreadsheet(english_data.fillna(''), 
                  sheet_id='17SxiHOwXKl6Iqf60JOJx5VCDUApfmiEl3PspvN84GHw',
                  styleClass=HysonFireStyle
                 )